# Bayesian Inference: Posterior Predictive

In [1]:
# Import some helper functions (please ignore this!)
from utils import *

**Context:** 

**Challenge:**

**Outline:**

**Overview.** TODO
* Our goal isn't just to learn the parameters; our goal is to determine, given observed data points, what's the probability of new data? State goal mathematically.
* Bayesian inference (sampling from the posterior) will actually help us accomplish that.
* To derive the distribution of new data given old data, we'll have to introduce a few more facts about directed graphical models.
* We'll instantiate everything here for regression, but the general principles apply

**Representing Unobserved Variables in DGMs.**
* TODO show for regression, circle instead of dot

**Representing the Joint Distribution of Training and Test Data in a DGM.**

**Laws of Conditional Independence.**

**Derivation.**

````{admonition} Exercise: Deriving the Posterior Predictive Distribution
TODO
````